<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Convert LaTeX Sentence to SymPy Expression

## Author: Ken Sible

## The following module will demonstrate a recursive descent parser for LaTeX.

### NRPy+ Source Code for this module:
1. [latex_parser.py](../edit/latex_parser.py); [\[**tutorial**\]](Tutorial-LaTeX_SymPy_Conversion.ipynb) The latex_parser.py script will convert a LaTeX sentence to a SymPy expression using the following function: parse(sentence).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#intro): Introduction: Lexical Analysis and Syntax Analysis
1. [Step 2](#sandbox): Demonstration and Sandbox (LaTeX Parser)
1. [Step 3](#tensor): Tensor Support with Einstein Notation
1. [Step 4](#latex_pdf_output): $\LaTeX$ PDF Output

<a id='intro'></a>

# Step 1: Lexical Analysis and Syntax Analysis \[Back to [top](#toc)\]
$$\label{intro}$$

In the following section, we discuss [lexical analysis](https://en.wikipedia.org/wiki/Lexical_analysis) (lexing) and [syntax analysis](https://en.wikipedia.org/wiki/Parsing) (parsing). In the process of lexical analysis, a lexer will tokenize a character string, called a sentence, using substring pattern matching (or tokenizing). We implemented a regex-based lexer for NRPy+, which does pattern matching using a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) for each token pattern. In the process of syntax analysis, a parser will receive a token iterator from the lexer and build a parse tree containing all syntactic information of the language, as specified by a [formal grammar](https://en.wikipedia.org/wiki/Formal_grammar). We implemented a [recursive descent parser](https://en.wikipedia.org/wiki/Recursive_descent_parser) for NRPy+, which will build a parse tree in [preorder](https://en.wikipedia.org/wiki/Tree_traversal#Pre-order_(NLR)), starting from the root [nonterminal](https://en.wikipedia.org/wiki/Terminal_and_nonterminal_symbols), using a [right recursive](https://en.wikipedia.org/wiki/Left_recursion) grammar. The following right recursive, [context-free grammar](https://en.wikipedia.org/wiki/Context-free_grammar) was written for parsing [LaTeX](https://en.wikipedia.org/wiki/LaTeX), adhering to the canonical (extended) [BNF](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form) notation used for describing a context-free grammar:

```
<ROOT>          -> <STRUCTURE> { <LINE_BREAK> <STRUCTURE> }*
<STRUCTURE>     -> <CONFIG> | <ENVIRONMENT> | <ASSIGNMENT>
<CONFIG>        -> '%' ( <DEFINE> | <EVAL> | <PARSE> | <REDEF> )
<ENVIRONMENT>   -> <BEGIN_ALIGN> <ASSIGNMENT> { <LINE_BREAK> <ASSIGNMENT> }* <END_ALIGN>
<ASSIGNMENT>    -> ( <TENSOR> | <COVDRV> ) = <EXPRESSION>
<EXPRESSION>    -> <TERM> { ( '+' | '-' ) <TERM> }*
<TERM>          -> <FACTOR> { [ '/' ] <FACTOR> }*
<FACTOR>        -> ( <BASE> | <EULER> ) { '^' <EXPONENT> }*
<BASE>          -> [ '-' ] ( <ATOM> | '(' <EXPRESSION> ')' )
<EXPONENT>      -> <BASE> | '{' <BASE> '}' | '{{' <BASE> '}}'
<ATOM>          -> <NUMBER> | <TENSOR> | <COMMAND> | <OPERATOR>
<NUMBER>        -> <RATIONAL> | <DECIMAL> | <INTEGER> | <PI>
<COMMAND>       -> <SQRT> | <FRAC> | <NLOG> | <TRIG>
    ⋮            ⋮
```

<small>**Source**: Robert W. Sebesta. Concepts of Programming Languages. Pearson Education Limited, 2016.</small>

In [1]:
from latex_parser import *
import sympy as sp

In [2]:
lexer = Lexer(); lexer.initialize(r'(1 + x/n)^n')
print(', '.join(token for token in lexer.tokenize()))

LEFT_PAREN, INTEGER, PLUS, LETTER, DIVIDE, LETTER, RIGHT_PAREN, CARET, LETTER


In [3]:
expr = parse_expr(r'(1 + x/n)^n')
print(expr, ':', sp.srepr(expr))

(1 + x/n)**n : Pow(Add(Integer(1), Mul(Pow(Symbol('n'), Integer(-1)), Symbol('x'))), Symbol('n'))


#### `Grammar Derivation: (1 + x/n)^n`
```
<EXPRESSION> -> <TERM>
             -> <FACTOR>
             -> <BASE>^<EXPONENT>
             -> (<EXPRESSION>)^<EXPONENT>
             -> (<TERM> + <TERM>)^<EXPONENT>
             -> (<FACTOR> + <TERM>)^<EXPONENT>
             -> (<BASE> + <TERM>)^<EXPONENT>
             -> (<ATOM> + <TERM>)^<EXPONENT>
             -> (<NUMBER> + <TERM>)^<EXPONENT>
             -> (<INTEGER> + <TERM>)^<EXPONENT>
             -> (1 + <TERM>)^<EXPONENT>
             -> (1 + <FACTOR> / <FACTOR>)^<EXPONENT>
             -> ...
```

<a id='sandbox'></a>

# Step 2: Demonstration and Sandbox (LaTeX Parser) \[Back to [top](#toc)\]
$$\label{sandbox}$$

We implemented a wrapper function for the `parse()` method that will accept a LaTeX sentence and return a SymPy expression. Furthermore, the entire parsing module was designed for extendibility. We apply the following procedure for extending parser functionality to include an unsupported LaTeX command: append that command to the grammar dictionary in the Lexer class with the mapping regex:token, write a grammar abstraction (similar to a regular expression) for that command, add the associated nonterminal (the command name) to the command abstraction in the Parser class, and finally implement the straightforward (private) method for parsing the grammar abstraction. We shall demonstrate the extension procedure using the `\sqrt` LaTeX command.

```<SQRT> -> <SQRT_CMD> [ '[' <INTEGER> ']' ] '{' <EXPRESSION> '}'```
```
def _sqrt(self):
    if self.accept('LEFT_BRACKET'):
        integer = self.lexer.lexeme
        self.expect('INTEGER')
        root = Rational(1, integer)
        self.expect('RIGHT_BRACKET')
    else: root = Rational(1, 2)
    self.expect('LEFT_BRACE')
    expr = self.__expr()
    self.expect('RIGHT_BRACE')
    if root == Rational(1, 2):
        return sqrt(expr)
    return Pow(expr, root)
```

In addition to expression parsing, we included support for equation parsing, which will produce a dictionary mapping LHS $\mapsto$ RHS, where LHS must be a symbol, and insert that mapping into the global namespace of the previous stack frame, as demonstrated below.

$$ \mathop{s_n} = \left(1 + \frac{1}{n}\right)^n $$

In [4]:
parse(r'\mathop{s_n} = \left(1 + \frac{1}{n}\right)^n')
print('s_n =', s_n)

s_n = (1 + 1/n)**n


We implemented robust error messaging using the custom `ParseError` exception, which should handle every conceivable case to identify, as detailed as possible, invalid syntax inside of a LaTeX sentence. The following are runnable examples of possible error messages (simply uncomment and run the cell):

In [5]:
# parse(r'\sqrt[*]{2}')
    # ParseError: \sqrt[*]{2}
    #                   ^
    # unexpected '*' at position 6

# parse(r'\sqrt[0.5]{2}')
    # ParseError: \sqrt[0.5]{2}
    #                   ^
    # expected token INTEGER at position 6

# parse(r'\command{x}')
    # ParseError: \command{x}
    #             ^
    # unsupported command '\command' at position 0

In [6]:
Parser.ignore_override(); Parser.namespace.clear()

In the sandbox code cell below, you can experiment with the LaTeX parser using the wrapper function `parse(sentence)`, where sentence must be a [raw string](https://docs.python.org/3/reference/lexical_analysis.html) to interpret a backslash as a literal character rather than an [escape sequence](https://en.wikipedia.org/wiki/Escape_sequence).

In [7]:
# Write Sandbox Code Here

<a id='tensor'></a>

# Step 3: Tensor Support with Einstein Notation (WIP) \[Back to [top](#toc)\]
$$\label{tensor}$$

In the following section, we demonstrate the current parser support for tensor notation using the Einstein summation convention. The first example will parse an equation for a tensor contraction, the second will parse an equation for raising an index using the metric tensor, and the third will parse an align enviroment with an equation dependency. In each example, every tensor should appear either on the LHS of an equation or inside of a configuration before appearing on the RHS of an equation. Moreover, the parser will raise an exception upon violation of the Einstein summation convention, i.e. an invalid free or bound index.

**Grammar for the Define Macro**

```
'define' [ <SYMMETRY> ] { <SYMBOL> }+ [ '(' <INTEGER> ')' ] { ',' [ <SYMMETRY> ] { <SYMBOL> }+ [ '(' <INTEGER> ')' ] }*
```

#### Example 1. [Tensor Contraction](https://en.wikipedia.org/wiki/Tensor_contraction)
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> h = h^\mu{}_\mu </pre> | $$ h = h^\mu{}_\mu $$

In [8]:
parse(r"""
    % define nosym hUD (4);
    h = h^\mu{}_\mu
""")

['hUD', 'h']

In [9]:
print('h =', h)

h = hUD00 + hUD11 + hUD22 + hUD33


#### Example 2. [Raising Tensor Index](https://en.wikipedia.org/wiki/Raising_and_lowering_indices)
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> v^\mu = g^{\mu\nu} v_\nu </pre> | $$ v^\mu = g^{\mu\nu} v_\nu $$

In [10]:
parse(r"""
    % define metric gUU (3), vD (3);
    v^\mu = g^{\mu\nu} v_\nu
""")

['gDD', 'gdet', 'gUU', 'vD', 'vU']

In [11]:
print('vU =', vU)

vU = [gUU00*vD0 + gUU01*vD1 + gUU02*vD2, gUU01*vD0 + gUU11*vD1 + gUU12*vD2, gUU02*vD0 + gUU12*vD1 + gUU22*vD2]


#### Example 3. [Permutation Symbol](https://en.wikipedia.org/wiki/Levi-Civita_symbol) ([Cross Product](https://en.wikipedia.org/wiki/Cross_product))
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> u_i = \epsilon_{ijk} v^j w^k </pre> | $$ u_i = \epsilon_{ijk} v^j w^k $$

In [12]:
parse(r"""
    % define permutation epsilonDDD (3);
    % define vU (3), wU (3);
    u_i = \epsilon_{ijk} v^j w^k
""")

['epsilonDDD', 'vU', 'wU', 'uD']

In [13]:
print('uD =', uD)

uD = [vU1*wU2 - vU2*wU1, -vU0*wU2 + vU2*wU0, vU0*wU1 - vU1*wU0]


#### Example 4 (1). [Four Current](https://en.wikipedia.org/wiki/Four-current) ([Covariant Derivative](https://en.wikipedia.org/wiki/Covariant_derivative))
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu} </pre> | $$ J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu} $$

The following are contextually inferred, dynamically generated, and injected into the global namespace for expansion of the covariant derivative $\nabla_\nu F^{\mu\nu}$
$$
\begin{align*}
    \Gamma^\mu_{ba} &= \frac{1}{2} g^{\mu c}(\partial_b\,g_{a c} + \partial_a\,g_{c b} - \partial_c\,g_{b a}) \\
    \Gamma^\nu_{ba} &= \frac{1}{2} g^{\nu c}(\partial_b\,g_{a c} + \partial_a\,g_{c b} - \partial_c\,g_{b a}) \\
    \nabla_a F^{\mu \nu} &= \partial_a F^{\mu \nu} + \Gamma^\mu_{b a} F^{b \nu} + \Gamma^\nu_{b a} F^{\mu b}
\end{align*}
$$

In [14]:
# parse(r"""
#     % define anti01 FUU (4), const k;
#     J^\mu = (4\pi k)^{-1} \nabla_\nu F^{\mu\nu}
# """)

#### Example 4 (2). Diacritic Support
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre lang="latex"> J^\mu = (4\pi k)^{-1} \hat{\nabla}_\nu F^{\mu\nu} </pre> | $$ J^\mu = (4\pi k)^{-1} \hat{\nabla}_\nu F^{\mu\nu} $$

In [15]:
# parse(r"""
#     % define anti01 FUU (4), const k;
#     J^\mu = (4\pi k)^{-1} \hat{\nabla}_\nu F^{\mu\nu}
# """)

#### Example 5. [Kretschmann Scalar](https://en.wikipedia.org/wiki/Kretschmann_scalar) ([Schwarzschild Metric](https://en.wikipedia.org/wiki/Schwarzschild_metric))
LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre><code>% define basis [t, r, \theta, \phi];<br>% define metric gDD (4), kronecker deltaDD (4);<br>% define const G, const M;<br>% parse g_{\mu\nu} = \delta_{\mu\nu};<br>&#92;begin{align}<br>&emsp;&emsp;&emsp;g_{0 0} &= -\left(1 - \frac{2GM}{r}\right) &#92;&#92;<br>&emsp;&emsp;&emsp;g_{1 1} &=  \left(1 - \frac{2GM}{r}\right)^{-1} &#92;&#92;<br>&emsp;&emsp;&emsp;g_{2 2} &= r^{{2}} &#92;&#92;<br>&emsp;&emsp;&emsp;g_{3 3} &= r^{{2}} \sin^2\theta<br>&#92;end{align}<br>% update metric gDD;|$$
% define basis [t, r, \theta, \phi];
% define metric gDD (4), kronecker deltaDD (4);
% define const G, const M;
% parse g_{\mu\nu} = \delta_{\mu\nu};
\begin{align}
    g_{0 0} &= -\left(1 - \frac{2GM}{r}\right) \\
    g_{1 1} &=  \left(1 - \frac{2GM}{r}\right)^{-1} \\
    g_{2 2} &= r^{{2}} \\
    g_{3 3} &= r^{{2}} \sin^2\theta
\end{align}
% update metric gDD;
$$

LaTeX Source | Rendered LaTeX
:----------- | :-------------
<pre><code>&#92;begin{align}<br>&emsp;&emsp;&emsp;R^\alpha{}_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu} + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} &#92;&#92;<br>&emsp;&emsp;&emsp;R^{\alpha\beta\mu\nu} &= g^{\beta a} g^{\mu b} g^{\nu c} R^\alpha_{a b c} &#92;&#92;<br>&emsp;&emsp;&emsp;R_{\alpha\beta\mu\nu} &= g_{\alpha a} R^a_{\beta\mu\nu} &#92;&#92;<br>&emsp;&emsp;&emsp;R &= R^{\alpha\beta\mu\nu} R_{\alpha\beta\mu\nu}<br>&#92;end{align}</code></pre>|$$
\begin{align}
    R^\alpha{}_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu} + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} \\
    R^{\alpha\beta\mu\nu} &= g^{\beta a} g^{\mu b} g^{\nu c} R^\alpha_{a b c} \\
    R_{\alpha\beta\mu\nu} &= g_{\alpha a} R^a_{\beta\mu\nu} \\
    K &= R^{\alpha\beta\mu\nu} R_{\alpha\beta\mu\nu}
\end{align}
$$

In [16]:
parse(r"""
    % define basis [t, r, \theta, \phi];
    % define metric gDD (4), kronecker deltaDD (4);
    % define const G, const M;
    % parse g_{\mu\nu} = \delta_{\mu\nu};
    \begin{align}
        g_{0 0} &= -\left(1 - \frac{2GM}{r}\right) \\
        g_{1 1} &=  \left(1 - \frac{2GM}{r}\right)^{-1} \\
        g_{2 2} &= r^{{2}} \\
        g_{3 3} &= r^{{2}} \sin^2\theta
    \end{align}
    % update metric gDD;
    \begin{align}
        R^\alpha{}_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu} + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} \\
        R^{\alpha\beta\mu\nu} &= g^{\beta a} g^{\mu b} g^{\nu c} R^\alpha_{a b c} \\
        R_{\alpha\beta\mu\nu} &= g_{\alpha a} R^a_{\beta\mu\nu} \\
        K &= R^{\alpha\beta\mu\nu} R_{\alpha\beta\mu\nu}
    \end{align}
""");

In [17]:
display(sp.simplify(K))

48*G**2*M**2/r**6

In [18]:
display(sp.Matrix(gDD))

Matrix([
[2*G*M/r - 1,                0,    0,                  0],
[          0, 1/(-2*G*M/r + 1),    0,                  0],
[          0,                0, r**2,                  0],
[          0,                0,    0, r**2*sin(theta)**2]])

In [19]:
display(sp.Matrix(GammaUDD[0][:][:]))

Matrix([
[                        0, -G*M/(r**2*(2*G*M/r - 1)), 0, 0],
[-G*M/(r**2*(2*G*M/r - 1)),                         0, 0, 0],
[                        0,                         0, 0, 0],
[                        0,                         0, 0, 0]])

In [20]:
display(sp.Matrix(GammaUDD[1][:][:]))

Matrix([
[G*M*(-2*G*M/r + 1)/r**2,                          0,                 0,                               0],
[                      0, -G*M/(r**2*(-2*G*M/r + 1)),                 0,                               0],
[                      0,                          0, -r*(-2*G*M/r + 1),                               0],
[                      0,                          0,                 0, -r*(-2*G*M/r + 1)*sin(theta)**2]])

In [21]:
display(sp.Matrix(GammaUDD[2][:][:]))

Matrix([
[0,   0,   0,                      0],
[0,   0, 1/r,                      0],
[0, 1/r,   0,                      0],
[0,   0,   0, -sin(theta)*cos(theta)]])

In [22]:
display(sp.Matrix(GammaUDD[3][:][:]))

Matrix([
[0,   0,                     0,                     0],
[0,   0,                     0,                   1/r],
[0,   0,                     0, cos(theta)/sin(theta)],
[0, 1/r, cos(theta)/sin(theta),                     0]])

The static variable `namespace` for the `Parser` class will provide access to the global namespace of the parser across each instance of the class.

In [23]:
# Parser.namespace # uncomment and run cell

We extended our robust error messaging using the custom `TensorError` exception, which should handle any inconsistent tensor dimension and any violation of the Einstein summation convention, specifically that a bound index must appear exactly once as a superscript and exactly once as a subscript in any single term and that a free index must appear in every term with the same position and cannot be summed over in any term. The following are runnable examples of possible error messages (simply uncomment and run the cell):

In [24]:
# parse(r"""
#     % h^{\mu\mu}_{\mu\mu} [4]: nosym;
#     h = h^{\mu\mu}_{\mu\mu}
# """)
    # TensorError: illegal bound index

# parse(r"""
#     % g^\mu_\nu [3]: sym01, v_\nu [3];
#     v^\mu = g^\mu_\nu v_\nu
# """)
    # TensorError: illegal bound index

# parse(r"""
#     % g^{\mu\nu} [3]: sym01, v_\mu [3], w_\nu [3];
#     u^\mu = g^{\mu\nu}(v_\mu + w_\nu)
# """)
    # TensorError: unbalanced free index

<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-LaTeX_SymPy_Conversion.pdf](Tutorial-LaTeX_SymPy_Conversion.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [25]:
# import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
# cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-LaTeX_SymPy_Conversion")

In [26]:
from latex_parser import *
expr = parse_expr(r'\partial_x (x^{{2}} + 2x)')
print(expr, '->', expr.doit())

Derivative(x**2 + 2*x, x) -> 2*x + 2


In [27]:
from latex_parser import *
Parser.ignore_override()
parse(r"""
    % define basis [x, y];
    % define vD (2);
    v_0 = x^{{2}} + 2x;
    v_1 = y\sqrt{x};
    T_{\mu\nu} = (\partial_\nu v_\mu)\,\partial^2_x (\ln(x))
""")
TDD

[[-(2*x + 2)/x**2, 0], [-y/(2*x**(5/2)), -1/x**(3/2)]]

In [28]:
from latex_parser import *
import sympy as sp
parse(r"""
    % define basis [\theta, \phi];
    % define metric gDD (2), kronecker deltaDD (2);
    % define const r;
    % parse g_{\mu\nu} = \delta_{\mu\nu};
    \begin{align*}
        g_{0 0} &= r^{{2}} \\
        g_{1 1} &= r^{{2}} \sin^2(\theta)
    \end{align*}
    % update metric gDD;
    \begin{align*}
        R^\alpha_{\beta\mu\nu} &= \partial_\mu \Gamma^\alpha_{\beta\nu} - \partial_\nu \Gamma^\alpha_{\beta\mu} + \Gamma^\alpha_{\mu\gamma}\Gamma^\gamma_{\beta\nu} - \Gamma^\alpha_{\nu\sigma}\Gamma^\sigma_{\beta\mu} \\
        R_{\alpha\beta\mu\nu} &= g_{\alpha a} R^a_{\beta\mu\nu} \\
        R_{\beta\nu} &= R^\alpha_{\beta\alpha\nu} \\
        R &= g^{\beta\nu} R_{\beta\nu}
    \end{align*}
""")
assert GammaUDD[0][1][1] == parse_expr(r'-\sin(\theta)\cos(\theta)')
assert GammaUDD[1][0][1] == GammaUDD[1][1][0] == parse_expr(r'\cos(\theta)/\sin(\theta)')
assert RDDDD[0][1][0][1] == -RDDDD[0][1][1][0] == -RDDDD[1][0][0][1] == RDDDD[1][0][1][0] == parse_expr(r'r^{{2}} \sin^2(\theta)')
assert RDD[0][0] == 1
assert RDD[1][1] == parse_expr(r'\sin^2(\theta)')
assert RDD[0][1] == RDD[1][0] == 0
assert R == parse_expr(r'2/r^{{2}}')